# Лабораторная работа. Ранжирование.

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.


### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

Загрузите данные конкурса [Интернет-математика 2009](http://imat2009.yandex.ru/datasets). Там же находится описание данных. Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание на формат данных: разбивать необходимо множество запросов, а не строчки датасета.

Ориентир: размер матрицы признаков обучающей выборки (67929, 245), тестовой - (29361, 245).

In [1]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

filename = 'imat2009_learning.txt'
with open(filename) as f:
    lines = f.readlines() 
    labes = []
    queries = {}
    data = np.reros((len(lines), 245), dtype=float.32) #пустая матрица из нулей
    for i, line in lines:
        
        row = line.split(' ')
        labels.append(row[0])
        query = row[-1]
        
        for feat in row(1:-2):
            feat_num = (feat.split(':'))[0]
            value = (feat.split(':'))[1]
            data[i,  feat_num-1] = value
        
        if not query in queries.key():
            queries[query] = [i]
        else:
            query.append(i)
        
    queries_trains, queries_test  = train_test_split(lines(queries.keys()), test_size = 0,3)
        
        
        break
        
    
    #CONTINUE
    

In [ ]:

x_train, x_test = data[train_indexes], data[test_indexes]
y_train, y_test = labels[train_indexes], labels[test_indexes]

labels = np.array(labels)

print(x_train.shape, len(y_train))
print(x_test.shape, len(y_test))

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам.

**Задание 1.** *Pointwise* подход. Воспользовавшись известными вам техниками построения линейной регрессии, обучите модель, предсказывающую оценку асессора.

In [3]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
clf.fit(x_train, y_train)

#CONTINUE

In [1]:
from metrics import ndcg_score

def nDCGQueries_score(y_predicted):
    score = []
    for query in queries_test:
        doc_ind = queries[query]
        doc_ind_in_test = [np.where(test_index==ind)[0][0] for ind in doc_ind]
        score.append(ndog_score[doc_ind], y_predicted[y_predicteed], )
        
    
    #nDCG по каждому запросу
    return #усредняем nDCG по всем запросам

###  Ранжируем с XGBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

**Задание 2.** Обучите модель **reg:linear**, в качестве метрики оценки качества (*eval_metric*) используя *nDCG*, а в качестве бустера решающее дерево. Настройте [параметры](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md) алгоритма и добейтесь высокого качества на тестовой выборке.

In [4]:
import xgboost as xgb

param = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1
}

#CONTINUE

/Users/kseniadrozdova/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Функция ранжирования Okapi BM25

Если у вас нет ассесоров, которые могут проставить оценки релевантности документам, вышепроделанный способ ранжирования вам не подходит. В этом случае можно использовать формулу *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)). Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где $f(q_i,D)$ - частота слова (TF) $q_i$ в документе $D$, $|D|$ - длина документа (количество слов в нём), а *avgdl* — средняя длина документа в коллекции. 
$$$$
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75.
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
где $N$ - общее количество документов в коллекции, а  $n(q_i)$ — количество документов, содержащих $q_i$. 

In [30]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0.0

def score_BM25(n, f, qf, r, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    everything_else = ((k + 1) * f) / (K + f)
    third = ((k2+1) * qf) / (k2 + qf)
    return IDF * second * third


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

**Задание по проекту.** 
Для его выполнения вам понадобится собранная коллекция документов и функция, составляющая обратный индекс по словам в коллекции.

Напишите функцию (или несколько отдельных логичный функций), которая по запросу $Q = q_1,..., g_n$ и коллекции $D$ сортирует выдачу подходящих документов. Будем считать документ подходящим, если он содержит хотя бы одно слово из запроса (из которого удалены стоп-слова). В качестве метрики используйте *Okapi BM25*.

Для проверки работы функции на вашем корпусе используйте запрос **каникулы на новый год и рождество**. Выведите ссылки в ipynb на первые десять докуменов в отсортированной выдаче(как во втором семинаре с помощью IPython.display) и их оценку BM25. Напомню, что ссылки на документы хрянятся в самих доках под тэгом @url.

Не забыть:
1. Лемматизировать запрос, удалить стоп-слова => запрос готов
2. Лемматизировать слова в документах => документы готовы к подсчетам статистик по запросу